<a href="https://colab.research.google.com/github/GDurgaGowri/Multimodal-sentimental-analysis-for-movie-review-system/blob/main/Multimodal_sentimental_analysis_deployment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [63]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [64]:
!pip install gradio torch scikit-learn


In [65]:
!pip install gradio torch torchvision torchaudio

In [66]:
import torch

# Load checkpoint
checkpoint = torch.load("drive/MyDrive/colab_models/sentiment_model.pth", map_location=torch.device("cpu"))

# Print keys in checkpoint
print(checkpoint.keys())

# If model_state_dict exists, print the LSTM layer shape
for key in checkpoint["model_state_dict"]:
    if "lstm" in key:
        print(f"{key}: {checkpoint['model_state_dict'][key].shape}")


dict_keys(['model_state_dict'])
lstm.weight_ih_l0: torch.Size([256, 128])
lstm.weight_hh_l0: torch.Size([256, 64])
lstm.bias_ih_l0: torch.Size([256])
lstm.bias_hh_l0: torch.Size([256])
lstm.weight_ih_l0_reverse: torch.Size([256, 128])
lstm.weight_hh_l0_reverse: torch.Size([256, 64])
lstm.bias_ih_l0_reverse: torch.Size([256])
lstm.bias_hh_l0_reverse: torch.Size([256])
lstm.weight_ih_l1: torch.Size([256, 128])
lstm.weight_hh_l1: torch.Size([256, 64])
lstm.bias_ih_l1: torch.Size([256])
lstm.bias_hh_l1: torch.Size([256])
lstm.weight_ih_l1_reverse: torch.Size([256, 128])
lstm.weight_hh_l1_reverse: torch.Size([256, 64])
lstm.bias_ih_l1_reverse: torch.Size([256])
lstm.bias_hh_l1_reverse: torch.Size([256])


<ipython-input-66-46a75a67d1e9>:4: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load("drive/MyDrive/colab_models/sentiment_model.pth", map_location=torch

In [67]:
model = SentimentAnalysisModel()
checkpoint = torch.load("drive/MyDrive/colab_models/sentiment_model.pth", map_location="cpu")

# Load state dict
model.load_state_dict(checkpoint["model_state_dict"])  # Should work now!


<ipython-input-67-4621ff6db6ec>:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load("drive/MyDrive/colab_models/sentiment_model.pth", map_location="cpu"

<All keys matched successfully>

In [68]:
model.eval()  # Model is ready for inference


SentimentAnalysisModel(
  (conv1): Conv1d(1, 64, kernel_size=(3,), stride=(1,), padding=(1,))
  (conv2): Conv1d(64, 128, kernel_size=(3,), stride=(1,), padding=(1,))
  (lstm): LSTM(128, 64, num_layers=2, batch_first=True, bidirectional=True)
  (fc1): Linear(in_features=128, out_features=1, bias=True)
)

In [69]:
import torch
import numpy as np
test_input = torch.randn(1, 100, 1)  # Correct shape (batch_size=1, channels=1, seq_length=100)
model.eval()

# Perform inference
with torch.no_grad():
    prediction = model(test_input)  # Model forward pass
    sentiment_score = torch.sigmoid(prediction)  # Convert logits to probability

# Print result
print(f"Predicted Sentiment Score: {sentiment_score.item()}")


Predicted Sentiment Score: 0.4946766495704651


In [70]:
import torch
import torch.nn as nn
import gradio as gr
import numpy as np
import re

class SentimentAnalysisModel(nn.Module):
    def __init__(self):
        super(SentimentAnalysisModel, self).__init__()

        self.conv1 = nn.Conv1d(in_channels=1, out_channels=64, kernel_size=3, padding=1)
        self.conv2 = nn.Conv1d(in_channels=64, out_channels=128, kernel_size=3, padding=1)

        self.lstm = nn.LSTM(
            input_size=128, hidden_size=64, num_layers=2,
            bidirectional=True, batch_first=True
        )

        self.fc1 = nn.Linear(128, 1)

    def forward(self, x):
        x = x.permute(0, 2, 1)  # Reshape for CNN (batch, channels, sequence)
        x = torch.relu(self.conv1(x))
        x = torch.relu(self.conv2(x))

        x = x.permute(0, 2, 1)  # Reshape for LSTM (batch, sequence, features)
        lstm_out, _ = self.lstm(x)

        out = self.fc1(lstm_out[:, -1, :])  # Last time-step output
        return out

# Load model checkpoint
model = SentimentAnalysisModel()
checkpoint = torch.load("drive/MyDrive/colab_models/sentiment_model.pth", map_location="cpu")
model.load_state_dict(checkpoint["model_state_dict"])
model.eval()

# Dummy vocabulary (replace with actual vocabulary)
dummy_vocab = {word: idx for idx, word in enumerate(["great", "bad", "amazing", "worst", "good", "terrible", "<pad>"])}

def preprocess_text(text, seq_length=100):
    text = re.sub(r"[^a-zA-Z0-9\s]", "", text.lower())  # Remove special characters
    tokens = text.split()

    # Pad or truncate to fixed length
    tokens = tokens[:seq_length] + ["<pad>"] * max(0, seq_length - len(tokens))

    # Convert words to indices using vocabulary
    numerical_input = [dummy_vocab.get(word, 0) for word in tokens]

    # Convert to tensor of shape (1, 100, 1)
    input_tensor = torch.tensor(numerical_input, dtype=torch.float32).view(1, 100, 1)
    return input_tensor

def predict_sentiment(text):
    try:
        input_tensor = preprocess_text(text)

        with torch.no_grad():
            prediction = model(input_tensor)
            sentiment_score = torch.sigmoid(prediction).item()

        if sentiment_score > 0.5:
            return f"Positive Sentiment 😊 (Score: {sentiment_score:.2f})"
        else:
            return f"Negative Sentiment 😠 (Score: {sentiment_score:.2f})"

    except Exception as e:
        return f"Error: {str(e)}"

# Gradio UI
gr.Interface(
    fn=predict_sentiment,
    inputs=gr.Textbox(lines=2, placeholder="Enter a movie review..."),
    outputs="text",
    title="Movie Review Sentiment Analysis",
    description="Enter a review to check if it's positive or negative!"
).launch()


<ipython-input-70-15e20645c047>:34: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load("drive/MyDrive/colab_models/sentiment_model.pth", map_location="cpu

Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://a283df31e0f52c2a7e.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
